# Part 3: LoRA Fine-tuning with NeMo

This notebook demonstrates how to fine-tune Llama 3.1 8B Instruct using LoRA (Low-Rank Adaptation) with NVIDIA NeMo framework.

## What is LoRA?

LoRA (Low-Rank Adaptation) is a parameter-efficient fine-tuning method that:
- Adds trainable low-rank matrices to frozen model weights
- Reduces memory requirements by 90%+
- Enables fine-tuning large models on consumer GPUs
- Produces small adapter files (~100-500MB for 8B models)

The focus of this workshop is not the specifics of LoRA, but to actually give everyone an guide on how to carry out the process of tuning your model. 

## IMPORTANT: NeMo Framework Setup

This notebook requires the NVIDIA NeMo framework for LoRA training. We'll use the cloned NeMo repository to access the necessary training scripts.

**NeMo Compatibility**: 
- The downloaded model uses standard NeMo format (.nemo file)
- The training scripts work directly without any modifications

**Training Experience**: In this workshop, you'll train your own LoRA adapter from scratch! This gives you hands-on experience with:
- Setting up training data
- Configuring LoRA parameters
- Running the actual training
- Testing your custom adapter

The training process takes approximately 5-10 minutes for our small example dataset.

## Clone NeMo Repo

This cell downloads the NVIDIA NeMo framework (if not already present) and verifies that the required training scripts are available.


In [ ]:
# Clone NeMo repository if not already present
import os

# Use relative path for NeMo
nemo_path = './NeMo'

if not os.path.exists(nemo_path):
    print("Cloning NeMo repository...")
    !git clone https://github.com/NVIDIA/NeMo.git {nemo_path}
    print("NeMo repository cloned successfully!")
else:
    print("NeMo repository already exists.")
    
# Verify the training scripts exist
nemo_scripts = [
    f'{nemo_path}/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py',
    f'{nemo_path}/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py',
    f'{nemo_path}/scripts/nlp_language_modeling/merge_lora_weights/merge.py'
]

print("\nChecking for required NeMo scripts:")
for script in nemo_scripts:
    if os.path.exists(script):
        print(f"✓ Found: {os.path.basename(script)}")
    else:
        print(f"✗ Missing: {script}")

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install jsonlines transformers omegaconf pytorch-lightning

This cell imports the required Python libraries for LoRA training and verifies that PyTorch can access the GPU, displaying the GPU name and available memory.

In [ ]:
import os
import json
import jsonlines
from omegaconf import OmegaConf
import torch

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Creating the directories needed

In [4]:
# Create directories
os.makedirs("lora_tutorial/data", exist_ok=True)
os.makedirs("lora_tutorial/models", exist_ok=True)
os.makedirs("lora_tutorial/configs", exist_ok=True)

## 2. Prepare Dataset

Time to create our training data! 

We're creating a customer support AI, but here's the key insight: we're not teaching it WHAT customer support is - Llama already knows that. We're teaching it HOW to do customer support in YOUR specific style.

Look at these examples:
- Notice the consistent, professional tone
- See how each response acknowledges the issue first
- Watch how we always offer a clear next step

With just 5 examples, we can dramatically change how the model responds.


In [ ]:
# Create a sample dataset for customer support fine-tuning
training_data = [
    {
        "input": "User: My order hasn't arrived yet. Order number is 12345.\\n\\nAssistant:",
        "output": "I apologize for the delay with your order #12345. Let me check the status for you right away. I'll need to verify some details first to ensure your privacy and security."
    },
    {
        "input": "User: How do I reset my password?\\n\\nAssistant:",
        "output": "I'd be happy to help you reset your password. For security, please click on 'Forgot Password' on the login page, enter your email address, and follow the instructions sent to your inbox."
    },
    {
        "input": "User: What is your return policy?\\n\\nAssistant:",
        "output": "Our return policy allows returns within 30 days of purchase with original receipt. Items must be in original condition with tags attached. Refunds are processed within 5-7 business days."
    },
    {
        "input": "User: I received a damaged product. What should I do?\\n\\nAssistant:",
        "output": "I'm sorry to hear you received a damaged product. Please take photos of the damage and packaging, then contact us with your order number. We'll arrange a replacement or refund immediately."
    },
    {
        "input": "User: Do you offer international shipping?\\n\\nAssistant:",
        "output": "Yes, we offer international shipping to over 50 countries. Shipping rates and delivery times vary by destination. You can check availability and costs at checkout."
    }
]

# Save training data
with jsonlines.open('lora_tutorial/data/train.jsonl', 'w') as writer:
    writer.write_all(training_data)

# Create validation data (smaller subset)
val_data = training_data[:2]
with jsonlines.open('lora_tutorial/data/val.jsonl', 'w') as writer:
    writer.write_all(val_data)

print(f"Created {len(training_data)} training examples")
print(f"Created {len(val_data)} validation examples")

### Verify Prerequisites Before Training

In [ ]:
# Verify prerequisites before training
import os
import glob

print("🔍 Checking prerequisites for training...\n")

# Check if NeMo is cloned - use relative path
nemo_path = "./NeMo"
if os.path.exists(nemo_path):
    print("✅ NeMo repository found")
else:
    print("❌ NeMo repository not found! Please run cell 2 to clone NeMo.")

# Check if training scripts exist
training_script = f"{nemo_path}/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py"
if os.path.exists(training_script):
    print("✅ Training script found")
else:
    print("❌ Training script not found!")

# Check if model is downloaded - look in subdirectories since NGC creates them
model_files = glob.glob("lora_tutorial/models/llama-3_1-8b-instruct/**/*.nemo", recursive=True)
if model_files:
    model_path = model_files[0]  # Use the first .nemo file found
    # Check if it's a complete model (>10GB)
    size_gb = os.path.getsize(model_path) / (1024**3)
    if size_gb > 10:
        print("✅ Llama 3.1 8B model found")
        print(f"\n📁 Model Information:")
        print(f"   Path: {model_path}")
        print(f"   Size: {size_gb:.2f} GB")
        print(f"   Format: Standard NeMo checkpoint (.nemo)")
    else:
        print(f"⚠️  Incomplete model found ({size_gb:.1f} GB)")
        print("   Please re-run the download in 00_Workshop_Setup.ipynb")
else:
    print("❌ Model not found! Please run notebook 00_Workshop_Setup.ipynb first")

# Check if training data exists
if os.path.exists("lora_tutorial/data/train.jsonl"):
    print("✅ Training data found")
else:
    print("❌ Training data not found! Please run the data preparation cells")

print("\n🎯 Ready to train!" if all([
    os.path.exists(nemo_path),
    os.path.exists(training_script),
    len(model_files) > 0 and os.path.getsize(model_files[0]) / (1024**3) > 10,  # Check complete model
    os.path.exists("lora_tutorial/data/train.jsonl")
]) else "\n⚠️ Please fix the issues above before training!")

## 3. Run LoRA Training

### Actually Run the Training! 🚀

This is the exciting part - you'll train your own LoRA adapter! 

**What will happen:**
1. The model will load from the .nemo checkpoint (takes ~30 seconds)
2. Training will run for 50 steps (~5-10 minutes)
3. Checkpoints will be saved every 25 steps
4. A final LoRA adapter will be exported as a .nemo file

**Note about warnings**: You'll see many warnings about missing configuration fields - these are normal and can be ignored. They appear because NeMo supports many optional features that aren't used in this training.

Let's train your custom model:

This cell runs the LoRA fine-tuning process, training Llama 3.1 8B on the customer support dataset for 50 steps using a single GPU with LoRA adapters.

In [ ]:
%%bash

# Actually run the LoRA training!

# Find the model file dynamically (NGC creates subdirectories)
MODEL_DIR="lora_tutorial/models/llama-3_1-8b-instruct"
MODEL=$(find "$MODEL_DIR" -name "*.nemo" -type f | head -1)

if [ -z "$MODEL" ]; then
    echo "ERROR: No .nemo model file found in $MODEL_DIR"
    echo "Please run notebook 00_Workshop_Setup.ipynb first to download the model"
    exit 1
fi

TRAIN_DS="[lora_tutorial/data/train.jsonl]"
VALID_DS="[lora_tutorial/data/val.jsonl]"

# Use relative path to NeMo
NEMO_PATH="./NeMo"

echo "✅ Found Llama 3.1 8B model at $MODEL"

# Run training with NeMo
torchrun --nproc_per_node=1 \
"${NEMO_PATH}/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py" \
    exp_manager.exp_dir=lora_tutorial/experiments \
    exp_manager.name=customer_support_lora \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    trainer.precision=bf16-mixed \
    trainer.val_check_interval=0.5 \
    trainer.max_steps=100 \
    model.megatron_amp_O2=True \
    ++model.mcore_gpt=True \
    model.tensor_model_parallel_size=1 \
    model.pipeline_model_parallel_size=1 \
    model.micro_batch_size=1 \
    model.global_batch_size=2 \
    model.restore_from_path=${MODEL} \
    model.data.train_ds.file_names=${TRAIN_DS} \
    model.data.train_ds.concat_sampling_probabilities=[1.0] \
    model.data.validation_ds.file_names=${VALID_DS} \
    model.peft.peft_scheme=lora \
    model.peft.lora_tuning.target_modules=[attention_qkv] \
    model.peft.lora_tuning.adapter_dim=32 \
    model.peft.lora_tuning.adapter_dropout=0.1 \
    model.optim.lr=5e-4

## 4. Verify Training Results

Look at what we've created! Let me break down these files:

1. **customer_support_lora.nemo** (21MB) - This is your golden ticket! Your custom AI adapter
2. **The .ckpt files** (147MB each) - Full training checkpoints with optimizer states


In [ ]:
# Check if training created the LoRA adapter
!ls -la ./lora_tutorial/experiments/customer_support_lora*/checkpoints/

## Summary

Congratulations! You've successfully:
- ✅ Set up the NeMo training environment
- ✅ Created training data for your custom task
- ✅ Configured LoRA parameters for efficient training
- ✅ Trained your own LoRA adapter on Llama 3.1 8B

Your LoRA adapter is now ready to be deployed with NVIDIA NIM in the next notebook!

**Next Steps**: 
- Open `04_Deploy_LoRA_with_NIM.ipynb` to deploy your custom model